In [75]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import datetime as dt

print('Imports completed!')

Imports completed!


In [76]:
# Read in game score data
game_df = pd.read_excel('./data/apiGameScores.xlsx')
game_df

,game_ids,date,visitor,home,visitor_points,home_points
0,319,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,1300,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,1344,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,1353,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,1376,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...,...
79,12202,2023-03-31T23:30:00.000Z,Atlanta Hawks,Brooklyn Nets,107.0,124.0
80,13399,2024-03-01T00:30:00.000Z,Atlanta Hawks,Brooklyn Nets,97.0,124.0
81,13414,2024-03-02T20:00:00.000Z,Atlanta Hawks,Brooklyn Nets,102.0,114.0
82,15092,2025-03-16T22:00:00.000Z,Atlanta Hawks,Brooklyn Nets,NaN,NaN


In [77]:
# Grab game ids to be used with api call
game_ids = game_df['game_ids'].values[:1]
game_ids

array([319], dtype=int64)

In [104]:
url = "https://api-nba-v1.p.rapidapi.com/games/statistics"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

statistics = []
for game_id in game_ids:
	querystring = {"id": game_id}
	response = requests.get(url, headers=headers, params=querystring)
	
 	# Check if the API request was successful
	if response.status_code == 200:
		game_statistics = response.json()
		# Extract game id
		id = game_statistics['parameters']['id']

		# Extract visitor and home team names
		visitor = game_statistics['response'][0]['team']['name']
		home = game_statistics['response'][1]['team']['name']

		# Extract statistics for visitor team
		visitor_statics = game_statistics['response'][0]['statistics'][0]
		visitor_stats = {f"visitor_{key}":value for key, value in visitor_statics.items()}
  
		visitor_stats['game_id'] = id
		visitor_stats['visitor_team'] = visitor
		
		# Extract statistics for home team
		home_statics = game_statistics['response'][1]['statistics'][0]
		home_stats = {f"home_{key}":value for key, value in home_statics.items()}
		
		home_stats['game_id'] = id
		home_stats['home_team'] = home

	visitor_stat_df = pd.DataFrame([visitor_stats])
	home_stat_df = pd.DataFrame([home_stats])


In [107]:
game_statistics_df = pd.concat([visitor_stat_df, home_stat_df], axis=1)
game_statistics_df

,visitor_fastBreakPoints,visitor_pointsInPaint,visitor_biggestLead,visitor_secondChancePoints,visitor_pointsOffTurnovers,visitor_longestRun,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,...,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,home_min,game_id,home_team
0,18,34,2,None,20,None,97,34,82,41.5,...,43,33,25,8,20,7,24,240:00,319,Atlanta Hawks
